In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv
/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip


In [2]:
!unzip -qq /kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
!unzip -qq /kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip

In [3]:
import os, shutil, pathlib

# Define the base directory
base_dir = pathlib.Path("/kaggle/working/")

# Define the source directory
original_dir = base_dir / "train"

# Define the destination directories
train_dir = base_dir / "train_organized" # Changed directory name for subset
validation_dir = base_dir / "validation_organized"
test_dir = base_dir / "test_organized"
print(train_dir)

/kaggle/working/train_organized


In [4]:
# Create the main destination directories
for directory in [train_dir, validation_dir, test_dir]:
    os.makedirs(directory, exist_ok=True)

# Create subdirectories for cats and dogs within each destination directory
for directory in [train_dir, validation_dir, test_dir]:
    os.makedirs(directory / "cats", exist_ok=True)
    os.makedirs(directory / "dogs", exist_ok=True)
# Function to get category from filename
def get_category(filename):
    if "cat" in filename:
        return "cats"
    elif "dog" in filename:
        return "dogs"
    else:
        return None

# Get list of all image files
all_files = list(original_dir.glob("*.jpg")) # Assuming all images are jpg

# Separate cat and dog files
cat_files = [f for f in all_files if "cat" in f.name]
dog_files = [f for f in all_files if "dog" in f.name]

# Shuffle files to create random train/validation/test splits
np.random.shuffle(cat_files)
np.random.shuffle(dog_files)
print("Done")

Done


In [5]:
# Define split sizes for each label in training
train_split = 7500
test_split = 5000     

train_cat_files = cat_files[:train_split]
train_dog_files = dog_files[:train_split]

test_cat_files = cat_files[train_split : train_split  + test_split]
test_dog_files = dog_files[train_split : train_split  + test_split]


# Function to copy files to their respective directories
def copy_files(file_list, destination_dir):
    for f in file_list:
        category = get_category(f.name)
        if category:
            shutil.copy(f, destination_dir / category / f.name)

# Copy files to train, validation, and test directories
print("Copying training files (subset)...")
copy_files(train_cat_files + train_dog_files, train_dir)
print("Copying test files...")
copy_files(test_cat_files + test_dog_files, test_dir)
print("Done")

Copying training files (subset)...
Copying test files...
Done


In [7]:
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (128, 128)
BATCH = 32

train = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True, 
    zoom_range = 0.1
).flow_from_directory(
    train_dir,
    target_size = IMG_SIZE,
    batch_size = BATCH,
    class_mode = 'binary', 
    shuffle = True
)

print("Class indices", train.class_indices)

2025-11-19 16:32:05.652665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763569925.878327      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763569925.948539      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Found 15000 images belonging to 2 classes.
Class indices {'cats': 0, 'dogs': 1}


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer = Adam(1e-3), loss = 'binary_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-11-19 16:32:38.180064: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
EPOCHS = 10
history = model.fit(train, epochs = EPOCHS)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 242s 510ms/step - accuracy: 0.5588 - loss: 0.6789
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 239s 509ms/step - accuracy: 0.7064 - loss: 0.5696
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 238s 506ms/step - accuracy: 0.7521 - loss: 0.5030
Epoch 4/10
221/469 ━━━━━━━━━━━━━━━━━━━━ 2:07 514ms/step - accuracy: 0.7841 - loss: 0.4655

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='binary',
    shuffle=False
)


In [ ]:
test_gen.reset()
y_prob = model.predict(test_gen).ravel()
y_pred = (y_prob >= 0.5).astype(int)
y_true = test_gen.classes

acc  = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec  = recall_score(y_true, y_pred)
f1   = f1_score(y_true, y_pred)

In [ ]:
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")



In [ ]:
model.save_weights('user_v2.weights.h5')

In [ ]:
metrics_data = {
    "model_name": "ClassificationModel",
    "timestamp": "2025-11-19T15:35:00Z",
    "dataset_used": "Dog and Cat",
    "metrics": {
        "accuracy": acc,
        "f1_score": f1,
        "precision" : prec,
        "Recall" : rec
    }
}

file_path = "metrics_v1.json"
with open(file_path, 'w') as f:
    json.dump(metrics_data, f, indent=4) # indent for pretty-printing